<a href="https://colab.research.google.com/github/abhiram1809/ML-Stuff/blob/main/Full_copy_Keyword_Extraction_and_Lexical_Simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-p95_7wlm
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-p95_7wlm
     |████████████████████████████████| 132 kB 6.2 MB/s 
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62565 sha256=8f632492e8087f9d9a7c6bccc918a990b1e48858610c6178969e23c6c7997b67
  Stored in directory: /tmp/pip-ephem-wheel-cache-iltc2hyc/wheels/52/79/f4/dae9309f60266aa3767a4381405002b6f2955fbcf038d804da
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73986 sha256=05fac7cb1fc23ef5d6b9711de8720ff21462e1719048f55f3e946f2636eae680
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built yake jellyfish


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
from functools import lru_cache
import re
import unicodedata
import sys
from collections import Counter
import nltk
nltk.download('brown')
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.tokenize import TextTilingTokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
import yake
from heapq import nsmallest
from operator import itemgetter

In [8]:
text = """A. INTRODUCTION TO OUR SERVICES

This Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple News+, Apple One, Apple Podcasts, Apple Podcast Subscriptions, Apple TV, Apple TV+, Apple TV Channels, Game Center, iTunes), through which you can buy, get, license, rent or subscribe to content, Apps (as defined below), and other in-app services (collectively, “Content”). Content may be offered through the Services by Apple or a third party. Our Services are available for your use in your country or territory of residence (“Home Country”). By creating an account for use of the Services in a particular country or territory you are specifying it as your Home Country. To use our Services, you need compatible hardware, software (latest version recommended and sometimes required) and Internet access (fees may apply). Our Services’ performance may be affected by these factors.

B. USING OUR SERVICES

PAYMENTS, TAXES, AND REFUNDS

You can acquire Content on our Services for free or for a charge, either of which is referred to as a “Transaction.” Each Transaction is an electronic contract between you and Apple, and/or you and the entity providing the Content on our Services. However, if you are a customer of Apple Distribution International Ltd., Apple Distribution International Ltd. is the merchant of record for some Content you acquire from Apple Books, Apple Podcasts, or App Store as displayed on the product page and/or during the acquisition process for the relevant Service. In such case, you acquire the Content from Apple Distribution International Ltd., which is licensed by the Content provider (e.g., App Provider (as defined below), book publisher, etc.). When you make your first Transaction, we will ask you to choose how frequently we should ask for your password for future Transactions. If you enable Touch ID for Transactions, we will ask you to authenticate all Transactions with your fingerprint, and if you enable Face ID for Transactions, we will ask you to authenticate all Transactions using facial recognition. Manage your password settings at any time by following these instructions: https://support.apple.com/HT204030.

Apple will charge your selected payment method (such as your credit card, debit card, gift card/code, or other method available in your Home Country) for any paid Transactions, including any applicable taxes. If you have also added it to your Apple Wallet, Apple may charge your selected payment method in Apple Wallet using Apple Pay. You can associate multiple payment methods with your Apple ID, and you agree that Apple may store and charge those payment methods for Transactions. Your primary payment method appears at the top of your account settings payments page.

If your primary payment method cannot be charged for any reason (such as expiration or insufficient funds), you authorize Apple to attempt to charge your other eligible payment methods in order from top to bottom as they appear on your account settings payments page.  If we cannot charge you, you remain responsible for any uncollected amounts, and we may attempt to charge you again or request that you provide another payment method. If you pre-order Content, you will be charged when the Content is delivered to you (unless you cancel prior to the Content’s availability). In accordance with local law, Apple may automatically update your payment information regarding your payment methods if provided such information by the payment networks or your financial institutions. For more details about how Transactions are billed, please visit http://support.apple.com/HT201359. All Transactions are final. Content prices may change at any time. If technical problems prevent or unreasonably delay delivery of Content, your exclusive and sole remedy is either replacement of the Content or refund of the price paid, as determined by Apple. From time to time, Apple may suspend or cancel payment or refuse a refund request if we find evidence of fraud, abuse, or unlawful or other manipulative behavior that entitles Apple to a corresponding counterclaim. Terms related to store credit and gift cards/codes are available here: https://www.apple.com/legal/internet-services/itunes/giftcards/.

ACCOUNT

Using our Services and accessing your Content may require an Apple ID. An Apple ID is the account you use across Apple’s ecosystem. Use of Game Center is subject to this Agreement and also requires a Game Center account. Your account is valuable, and you are responsible for maintaining its confidentiality and security. Apple is not responsible for any losses arising from the unauthorized use of your account. Please contact Apple if you suspect that your account has been compromised.

You must be age 13 (or equivalent minimum age in your Home Country, as set forth in the registration process) to create an account and use our Services. Apple IDs for persons under this age can be created by a parent or legal guardian using Family Sharing or by an approved educational institution. A parent or legal guardian who is creating an account for a child under the age of majority should review this Agreement with the child to ensure that they both understand it.

You may add, notify, or remove a Legacy Contact for your Apple ID as described in http://support.apple.com/HT212360. A Legacy Contact’s access to your Apple ID is limited as described in http://support.apple.com/HT212361.

PRIVACY

Your use of our Services is subject to Apple’s Privacy Policy, which is available at https://www.apple.com/legal/privacy/.

SERVICES AND CONTENT USAGE RULES

Your use of the Services and Content must follow the rules set forth in this section (“Usage Rules”). Any other use of the Services and Content is a material breach of this Agreement. Apple may monitor your use of the Services and Content to ensure that you are following these Usage Rules.

All Services:

- You may use the Services and Content only for personal, noncommercial purposes (except as set forth in the App Store Content section below).

- Apple’s delivery of Services or Content does not transfer any commercial or promotional use rights to you, and does not constitute a grant or waiver of any rights of the copyright owners.

- You can use Content from up to five different Apple IDs on each device.

- For any Service, you can have up to 10 devices (but only a maximum of 5 computers) signed in with your Apple ID at one time, though simultaneous streams or downloads of Content may be limited to a lower number of devices as set out below under Apple Music and Apple TV content. Each computer must also be authorized using the same Apple ID (to learn more about authorization of computers, visit https://support.apple.com/HT201251). Devices can be associated with a different Apple ID once every 90 days.

- Manipulating play counts, downloads, ratings, or reviews via any means — such as (i) using a bot, script, or automated process; or (ii) providing or accepting any kind of compensation or incentive — is prohibited.

- It is your responsibility not to lose, destroy, or damage Content once downloaded. We encourage you to back up your Content regularly.

- You may not tamper with or circumvent any security technology included with the Services.

- You may access our Services only using Apple’s software, and may not modify or use modified versions of such software.

- Video Content requires an HDCP connection.

Audio and Video Content Sales and Rentals:

- You can use Digital Rights Management (DRM)-free Content on a reasonable number of compatible devices that you own or control. DRM-protected Content can be used on up to five computers and any number of devices that you sync to from those computers.

- Content rentals are viewable on a single device at a time, and must be played within 30 days, and completed within 48 hours of the start of play (stopping, pausing or restarting does not extend this period).

- You may burn an audio playlist of purchased music to disc for listening purposes up to seven times; this limitation does not apply to DRM-free Content. Other Content may not be burned to disc.

- Purchased Content will generally remain available for you to download, redownload, or otherwise access from Apple. Though it is unlikely, subsequent to your purchase, Content may be removed from the Services (for instance, because the provider removed it) and become unavailable for further download or access from Apple. To ensure your ability to continue enjoying Content, we encourage you to download all purchased Content to a device in your possession and to back it up.

App Store Content:

- The term “Apps” includes apps and App Clips for any Apple platform and/or operating system, including any in-app purchases, extensions (such as keyboards), stickers, and subscriptions made available in such apps or App Clips.

- Individuals acting on behalf of a commercial enterprise, governmental organization or educational institution (an “Enterprise”) may download and sync non-Arcade Apps for use by either (i) a single individual on one or more devices owned or controlled by an Enterprise; or (ii) multiple individuals on a single shared device owned or controlled by an Enterprise. For the sake of clarity, each device used serially or collectively by multiple users requires a separate license.

Apple Music:

- An Individual Apple Music membership allows you to stream on a single device at a time; a Family membership allows you or your Family members to stream on up to six devices at a time.

Apple Arcade:

- Apple Arcade Apps may only be downloaded, or redownloaded, with a valid Apple Arcade trial or subscription.

- If your subscription ends, Apps downloaded via Apple Arcade will no longer be accessible to you.

Apple TV Content:

- For most channels, you can stream Content on up to three devices simultaneously.

- Learn more about Apple TV Content Usage Rules at https://support.apple.com/HT210074.

DOWNLOADS

You may be limited in the amount of Content you may download, and some downloaded Content may expire after a given amount of time after downloaded or first played. Certain Content may not be available for download at all.

You may be able to redownload previously acquired Content (“Redownload”) to your devices that are signed in with the same Apple ID (“Associated Devices”). You can see Content types available for Redownload in your Home Country at https://support.apple.com/HT204632. Content may not be available for Redownload if that Content is no longer offered on our Services.

Content also may be removed from our Services at any time, after which it cannot be downloaded, redownloaded, or otherwise accessed from Apple.

SUBSCRIPTIONS

The Services and certain Apps may allow you to purchase access to Content or Services on a subscription basis (“Paid Subscriptions”). Paid Subscriptions automatically renew until cancelled in the Manage Subscriptions section of your account settings. To learn more about cancelling your subscriptions, visit https://support.apple.com/HT202039. We will notify you if the price of a Paid Subscription increases and, if required, seek your consent to continue. You will be charged no more than 24 hours prior to the start of the latest Paid Subscription period. If we cannot charge your payment method for any reason (such as expiration or insufficient funds), and you have not cancelled the Paid Subscription, you remain responsible for any uncollected amounts, and we will attempt to charge the payment method as you may update your payment method information. This may result in a change to the start of your next Paid Subscription period and may change the date on which you are billed for each period. We reserve the right to cancel your Paid Subscription if we are unable to successfully charge your payment method to renew your subscription. Certain Paid Subscriptions may offer a free trial prior to charging your payment method. If you decide to unsubscribe from a Paid Subscription before we start charging your payment method, cancel the subscription at least 24 hours before the free trial ends.

If you start a free trial to a Paid Subscription offered by Apple as Content provider (an “Apple Paid Subscription”) and cancel before it ends, you cannot reactivate the free trial.

Free trials or free offers to Apple Paid Subscriptions, excluding iCloud, cannot be combined with any free trials or offers of Apple One. If you are in a free trial or free offer for any Apple Paid Subscriptions, and you subscribe to Apple One, your free trial(s) or offer(s) will not be paused even if you have access to such Apple Paid Subscription(s) through your Apple One subscription. You acknowledge that your free trial or free offer may expire while you are a Paid Subscriber to Apple One, and Apple shall have no obligation to reinstate, reimburse, or otherwise compensate you for any part of such expired free trial or free offer.

When your Paid Subscription to any Service or Content ends, you will lose access to any functionality or Content of that Service that requires a Paid Subscription.

CONTENT AND SERVICE AVAILABILITY

Terms found in this Agreement that relate to Services, Content types, features or functionality not available in your Home Country are not applicable to you unless and until they become available to you. To see the Content types available to you in your Home Country, go to the Services or visit https://support.apple.com/HT204411. Certain Services and Content available to you in your Home Country may not be available to you when traveling outside of your Home Country.

THIRD-PARTY DEVICES AND EQUIPMENT

You may not be able to use all features of the Services when accessing them on a non-Apple-branded device. Additionally, certain Services may require, direct, or suggest you use third-party equipment in some circumstances and/or for certain activities; such use is subject to the terms and conditions of such equipment and should be made in accordance with the applicable manufacturer’s instructions. By using the Services, you agree that Apple may automatically download and install minor updates to its software on third-party equipment from time to time.

C. YOUR SUBMISSIONS TO OUR SERVICES

Our Services may allow you to submit or post materials such as comments, ratings and reviews, pictures, videos, and podcasts (including associated metadata and artwork). Your use of such features must comply with the Submissions Guidelines below, which may be updated from time to time, and if we become aware of materials that violate our Submission Guidelines we will remove them. If you see materials that do not comply with the Submissions Guidelines, please use the Report a Concern feature. Except to the extent prohibited by law, you hereby grant Apple a worldwide, royalty-free, perpetual, nonexclusive license to use the materials you submit within the Services and related marketing, and Apple internal purposes. Apple may monitor and decide to remove or edit any submitted material.

Submissions Guidelines: You may not use the Services to:

- post any materials that (i) you do not have permission, right or license to use, or (ii) infringe on the rights of any third party;

- post objectionable, offensive, unlawful, deceptive, inaccurate, or harmful content;

- post personal, private or confidential information belonging to others;

- request personal information from a minor;

- impersonate or misrepresent your affiliation with another person, or entity;

- post or transmit spam, including but not limited to unsolicited or unauthorized advertising, promotional materials, or informational announcements;

- post, modify, or remove a rating or review in exchange for any kind of compensation or incentive;

- post a dishonest, abusive, harmful, misleading, or bad-faith rating or review, or a rating or review that is irrelevant to the Content being reviewed;

- plan or engage in any illegal, fraudulent, or manipulative activity.

D. FAMILY SHARING

The organizer of a Family (“Organizer”) must be 18 years or older and the parent or legal guardian of any Family member under age 13 or the equivalent minimum age in their Home Country (as set forth in the registration process). Apple devices are required for access to all of the Family Sharing features.

Purchase Sharing: Family Sharing’s Purchase Sharing feature allows eligible Content to be shared among up to six members of a Family. The Organizer invites other members to participate, and agrees to pay for all Transactions initiated by Family members. The Organizer's eligible payment methods are used to pay for any Transaction initiated by a Family member (except when the Family member’s account has store credit, which is always used first). Family members are acting as agents for the Organizer when the Organizer’s eligible payment methods are used. The Organizer hereby agrees (1) to pay for such Transactions; (2) that Transactions initiated by Family members are authorized; and (3) Transactions will be charged to eligible payment methods in the manner indicated in Section B above. Organizers are responsible for complying with their payment method contracts, and assume all risk related to sharing access to their eligible payment methods with Family members. A receipt or invoice for any Family member Transaction is sent to the initiating Family member and the Organizer.

Ask to Buy: Ask to Buy is a convenient feature that allows an Organizer to approve Transactions initiated by a Family member under age 18 (or the equivalent age of majority in your Home Country). The Organizer must be the parent or legal guardian of any Family member for whom Ask to Buy is activated. Content shared by Family members or acquired via content codes may not be subject to Ask to Buy.

Family Member changes: When a Family member leaves or is removed from the Family, the remaining Family members may no longer be able to access the former member’s Content, including Content acquired with the Organizer’s payment method.

Family Sharing Rules: You can only belong to one Family at a time, and may join any Family no more than twice per year. You can change the Apple ID you associate with a Family no more than once every 90 days. All Family members must share the same Home Country. Not all Content, including In-App Purchases, subscriptions, and some previously acquired Apps, are eligible for Purchase Sharing. Apple TV+, Apple TV Channels, Apple One Family, Apple One Premier, Apple Music Family, Apple Arcade, Apple News+, and Apple Fitness+ subscriptions are automatically enabled for Family Sharing. Subscriptions shared by a Family may be subject to Content usage limitations on a per subscription basis.

E. PERSONALIZED RECOMMENDATION FEATURES

The Services may recommend Content to you based on your downloads, purchases and other activities. You may opt out from receiving such personalized recommendations for some Services in your account settings.

F. ADDITIONAL ITUNES STORE TERMS

SEASON PASS AND MULTI-PASS

A Pass allows you to purchase and receive television Content as it becomes available. A Season Pass applies to television Content that has a limited number of episodes per season; a Multi-Pass applies to television Content that is available on an ongoing basis. The full price of a Season Pass or Multi-Pass is charged at the time of the Transaction. Season Pass or Multi-Pass Content is available for download up to 90 days after the last episode becomes available. If automatic renewal is selected when you obtain a Multi-Pass, you will be charged the full price of each subsequent Multi-Pass cycle. You can turn off automatic renewal at least 24 hours prior to the beginning of the next Multi-Pass cycle in your account settings. If a Content provider delivers to Apple fewer TV episodes than planned when you purchased a Season Pass, we will credit to your Apple ID the retail value of the corresponding number of episodes that were not provided to Apple.

G. ADDITIONAL APP STORE TERMS (EXCLUDING APPLE ARCADE APPS)

LICENSE OF APP STORE CONTENT

App licenses are provided to you by Apple or a third party developer (“App Provider”). If you are a customer of Apple Distribution International Ltd., the merchant of record is Apple Distribution International Ltd., which means that you acquire the App license from Apple Distribution International Ltd., but the App is licensed by the App Provider. An App licensed by Apple is an “Apple App;” an App licensed by an App Provider is a “Third Party App.” Apple acts as an agent for App Providers in providing the App Store and is not a party to the sales contract or user agreement between you and the App Provider. Any App that you acquire is governed by the Licensed Application End User License Agreement (“Standard EULA”) set forth below, unless Apple or the App Provider provides an overriding custom license agreement (“Custom EULA”). The App Provider of any Third Party App is solely responsible for its content, warranties, and claims that you may have related to the Third Party App. You acknowledge and agree that Apple is a third-party beneficiary of the Standard EULA or Custom EULA applicable to each Third Party App and may therefore enforce such agreement. Certain Apps, such as stickers and iMessage apps, may not appear on the device springboard but can be accessed and used in the Messages app drawer.

IN-APP PURCHASES

Apps may offer content, services or functionality for use within such Apps (“In-App Purchases”). In-App Purchases that are consumed during the use of the App (for example, virtual gems) cannot be transferred among devices and can be downloaded only once. You must authenticate your account before making In-App Purchases – separate from any authentication to obtain other Content – by entering your password or using Touch ID or Face ID. You will be able to make additional In-App Purchases for fifteen minutes without re-authenticating unless you’ve asked us to require a password for every purchase or have enabled Touch ID or Face ID. You can turn off the ability to make In-App Purchases by following these instructions: https://support.apple.com/HT201304.

APP MAINTENANCE AND SUPPORT

Apple is responsible for providing maintenance and support for Apple Apps only, or as required under applicable law. App Providers are responsible for providing maintenance and support for Third Party Apps.

APP BUNDLES

Some Apps may be sold together as a bundle (“App Bundle”). The price displayed with an App Bundle is the price you will be charged upon purchasing the App Bundle. The App Bundle price may be reduced to account for Apps you have already purchased or acquired, but may include a minimum charge to complete the App Bundle.

LICENSED APPLICATION END USER LICENSE AGREEMENT

Apps made available through the App Store are licensed, not sold, to you. Your license to each App is subject to your prior acceptance of either this Licensed Application End User License Agreement (“Standard EULA”), or a custom end user license agreement between you and the Application Provider (“Custom EULA”), if one is provided. Your license to any Apple App under this Standard EULA or Custom EULA is granted by Apple, and your license to any Third Party App under this Standard EULA or Custom EULA is granted by the Application Provider of that Third Party App. Any App that is subject to this Standard EULA is referred to herein as the “Licensed Application.” The Application Provider or Apple as applicable (“Licensor”) reserves all rights in and to the Licensed Application not expressly granted to you under this Standard EULA.

a. Scope of License: Licensor grants to you a nontransferable license to use the Licensed Application on any Apple-branded products that you own or control and as permitted by the Usage Rules. The terms of this Standard EULA will govern any content, materials, or services accessible from or purchased within the Licensed Application as well as upgrades provided by Licensor that replace or supplement the original Licensed Application, unless such upgrade is accompanied by a Custom EULA. Except as provided in the Usage Rules, you may not distribute or make the Licensed Application available over a network where it could be used by multiple devices at the same time. You may not transfer, redistribute or sublicense the Licensed Application except as expressly permitted in this Agreement and, if you sell your Apple Device to a third party, you must remove the Licensed Application from the Apple Device before doing so. You may not copy (except as permitted by this license and the Usage Rules), reverse-engineer, disassemble, attempt to derive the source code of, modify, or create derivative works of the Licensed Application, any updates, or any part thereof (except as and only to the extent that any foregoing restriction is prohibited by applicable law or to the extent as may be permitted by the licensing terms governing use of any open-sourced components included with the Licensed Application).

b. Consent to Use of Data: You agree that Licensor may collect and use technical data and related information—including but not limited to technical information about your device, system and application software, and peripherals—that is gathered periodically to facilitate the provision of software updates, product support, and other services to you (if any) related to the Licensed Application. Licensor may use this information, as long as it is in a form that does not personally identify you, to improve its products or to provide services or technologies to you.

c. Termination. This Standard EULA is effective until terminated by you or Licensor. Your rights under this Standard EULA will terminate automatically if you fail to comply with any of its terms.

d. External Services. The Licensed Application may enable access to Licensor’s and/or third-party services and websites (collectively and individually, "External Services"). You agree to use the External Services at your sole risk. Licensor is not responsible for examining or evaluating the content or accuracy of any third-party External Services, and shall not be liable for any such third-party External Services. Data displayed by any Licensed Application or External Service, including but not limited to financial, medical and location information, is for general informational purposes only and is not guaranteed by Licensor or its agents. You will not use the External Services in any manner that is inconsistent with the terms of this Standard EULA or that infringes the intellectual property rights of Licensor or any third party. You agree not to use the External Services to harass, abuse, stalk, threaten or defame any person or entity, and that Licensor is not responsible for any such use. External Services may not be available in all languages or in your Home Country, and may not be appropriate or available for use in any particular location. To the extent you choose to use such External Services, you are solely responsible for compliance with any applicable laws. Licensor reserves the right to change, suspend, remove, disable or impose access restrictions or limits on any External Services at any time without notice or liability to you.

e. NO WARRANTY: YOU EXPRESSLY ACKNOWLEDGE AND AGREE THAT USE OF THE LICENSED APPLICATION IS AT YOUR SOLE RISK. TO THE MAXIMUM EXTENT PERMITTED BY APPLICABLE LAW, THE LICENSED APPLICATION AND ANY SERVICES PERFORMED OR PROVIDED BY THE LICENSED APPLICATION ARE PROVIDED "AS IS" AND “AS AVAILABLE,” WITH ALL FAULTS AND WITHOUT WARRANTY OF ANY KIND, AND LICENSOR HEREBY DISCLAIMS ALL WARRANTIES AND CONDITIONS WITH RESPECT TO THE LICENSED APPLICATION AND ANY SERVICES, EITHER EXPRESS, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES AND/OR CONDITIONS OF MERCHANTABILITY, OF SATISFACTORY QUALITY, OF FITNESS FOR A PARTICULAR PURPOSE, OF ACCURACY, OF QUIET ENJOYMENT, AND OF NONINFRINGEMENT OF THIRD-PARTY RIGHTS. NO ORAL OR WRITTEN INFORMATION OR ADVICE GIVEN BY LICENSOR OR ITS AUTHORIZED REPRESENTATIVE SHALL CREATE A WARRANTY. SHOULD THE LICENSED APPLICATION OR SERVICES PROVE DEFECTIVE, YOU ASSUME THE ENTIRE COST OF ALL NECESSARY SERVICING, REPAIR, OR CORRECTION. SOME JURISDICTIONS DO NOT ALLOW THE EXCLUSION OF IMPLIED WARRANTIES OR LIMITATIONS ON APPLICABLE STATUTORY RIGHTS OF A CONSUMER, SO THE ABOVE EXCLUSION AND LIMITATIONS MAY NOT APPLY TO YOU.

f. Limitation of Liability. TO THE EXTENT NOT PROHIBITED BY LAW, IN NO EVENT SHALL LICENSOR BE LIABLE FOR PERSONAL INJURY OR ANY INCIDENTAL, SPECIAL, INDIRECT, OR CONSEQUENTIAL DAMAGES WHATSOEVER, INCLUDING, WITHOUT LIMITATION, DAMAGES FOR LOSS OF PROFITS, CORRUPTION OR LOSS OF DATA, FAILURE TO TRANSMIT OR RECEIVE ANY DATA (INCLUDING WITHOUT LIMITATION COURSE INSTRUCTIONS, ASSIGNMENTS AND MATERIALS), BUSINESS INTERRUPTION, OR ANY OTHER COMMERCIAL DAMAGES OR LOSSES, ARISING OUT OF OR RELATED TO YOUR USE OF OR INABILITY TO USE THE LICENSED APPLICATION, HOWEVER CAUSED, REGARDLESS OF THE THEORY OF LIABILITY (CONTRACT, TORT, OR OTHERWISE) AND EVEN IF LICENSOR HAS BEEN ADVISED OF THE POSSIBILITY OF SUCH DAMAGES. SOME JURISDICTIONS DO NOT ALLOW THE EXCLUSION OR LIMITATION OF LIABILITY FOR PERSONAL INJURY, OR OF INCIDENTAL OR CONSEQUENTIAL DAMAGES, SO THIS LIMITATION MAY NOT APPLY TO YOU. In no event shall Licensor’s total liability to you for all damages (other than as may be required by applicable law in cases involving personal injury) exceed the amount of two hundred and fifty dollars ($250.00). The foregoing limitations will apply even if the above stated remedy fails of its essential purpose.

g. You may not use or otherwise export or re-export the Licensed Application except as authorized by United States law and the laws of the jurisdiction in which the Licensed Application was obtained. In particular, but without limitation, the Licensed Application may not be exported or re-exported (a) into any U.S.-embargoed countries or (b) to anyone on the U.S. Treasury Department's Specially Designated Nationals List or the U.S. Department of Commerce Denied Persons List or Entity List. By using the Licensed Application, you represent and warrant that you are not located in any such country or on any such list. You also agree that you will not use these products for any purposes prohibited by United States law, including, without limitation, the development, design, manufacture, or production of nuclear, missile, or chemical or biological weapons.

h. U.S. Government End Users. The Licensed Application and related documentation are "Commercial Items", as that term is defined at 48 C.F.R. §2.101, consisting of "Commercial Computer Software" and "Commercial Computer Software Documentation", as such terms are used in 48 C.F.R. §12.212 or 48 C.F.R. §227.7202, as applicable. Consistent with 48 C.F.R. §12.212 or 48 C.F.R. §227.7202-1 through 227.7202-4, as applicable, the Commercial Computer Software and Commercial Computer Software Documentation are being licensed to U.S. Government end users (a) only as Commercial Items and (b) with only those rights as are granted to all other end users pursuant to the terms and conditions herein. Unpublished-rights reserved under the copyright laws of the United States.

i. Except to the extent expressly provided in the following paragraph, this Agreement and the relationship between you and Apple shall be governed by the laws of the State of California, excluding its conflicts of law provisions. You and Apple agree to submit to the personal and exclusive jurisdiction of the courts located within the county of Santa Clara, California, to resolve any dispute or claim arising from this Agreement. If (a) you are not a U.S. citizen; (b) you do not reside in the U.S.; (c) you are not accessing the Service from the U.S.; and (d) you are a citizen of one of the countries identified below, you hereby agree that any dispute or claim arising from this Agreement shall be governed by the applicable law set forth below, without regard to any conflict of law provisions, and you hereby irrevocably submit to the non-exclusive jurisdiction of the courts located in the state, province or country identified below whose law governs:

If you are a citizen of any European Union country or the United Kingdom, Switzerland, Norway or Iceland, the governing law and forum shall be the laws and courts of your usual place of residence.

Specifically excluded from application to this Agreement is that law known as the United Nations Convention on the International Sale of Goods.

H. ADDITIONAL TERMS FOR CERTAIN CONTENT ACQUIRED FROM THIRD PARTIES

Some Content available in Apple Books, App Store, and Apple Podcasts (including Apple Podcast Subscriptions) are acquired by You from the third-party provider of such Content (as displayed on the product page and/or during the acquisition process for the relevant Content), not Apple. For example, Apple Books Content is acquired from book publishers, not Apple. In such case, Apple acts as an agent for the Content provider in providing the Content to you, and therefore Apple is not a party to the Transaction between you and the Content provider. However, if you are a customer of Apple Distribution International Ltd., Apple Distribution International Ltd. is the merchant of record for the Content you acquire, but such Content is licensed by the Content provider. The Content provider reserves the right to enforce the terms of use relating to such Content. The Content provider is solely responsible for such Content, any warranties to the extent that such warranties have not been disclaimed, and any claims that you or any other party may have relating to such Content.

I. ADDITIONAL APPLE MUSIC TERMS

iCloud Music Library is an Apple Music feature that allows you to access your matched or uploaded songs, playlists and music videos acquired from Apple Music, the iTunes Store or another source (“iCloud Music Library Content”) on your Apple Music-enabled devices. iCloud Music Library is turned on automatically when you set up your Apple Music membership. iCloud Music Library collects information about your iCloud Music Library Content. This information is associated with your Apple ID, and compared to iCloud Music Library Content currently available on Apple Music. iCloud Music Library Content that is not matched is uploaded to Apple’s iCloud Music Library servers (in a format determined by Apple). You can upload up to 100,000 songs. Songs acquired from the iTunes Store do not count against this limit. Songs that do not meet certain criteria (for example, excessively large files) or that are not authorized for your device are not eligible for iCloud Music Library. When you use iCloud Music Library, Apple logs information such as the tracks you play, stop or skip, the devices you use, and the time and duration of playback. You agree to use iCloud Music Library only for lawfully acquired content. iCloud Music Library is provided on an “AS IS” basis and could contain errors or inaccuracies. You should back up your data and information prior to using iCloud Music Library. If you are not an Apple Music member, you may purchase an iTunes Match subscription, which uses iCloud Music Library. When your Apple Music membership ends, you will lose access to your iCloud Music Library, including iCloud Music Library Content that is uploaded to iCloud Music Library servers.

J. ADDITIONAL APPLE FITNESS+ TERMS

Apple Fitness+ is for entertainment and/or informational purposes only and is not intended to provide any medical advice. You should always seek the advice of an appropriately qualified healthcare professional regarding (a) the safety and advisability of any given activity, or (b) any specific medical condition or symptoms.

K. CARRIER MEMBERSHIP

Where available, you may be offered to purchase a Service membership from your wireless carrier (a “Carrier Membership”). If you purchase a Carrier Membership, your carrier is the merchant of record, which means that you acquire the Service license from your carrier, which will bill you for the cost of your Service membership, but the Service is licensed and provided by Apple. Your purchase relationship with the carrier is governed by the carrier’s terms and conditions, not this Agreement, and any billing disputes related to a Carrier Membership must be directed to your carrier, not Apple. By using a Service through a Carrier Membership, you agree that your carrier may exchange your carrier account information, telephone number and subscription information with Apple, and that Apple may use this information to determine the status of your Carrier Membership.

L. MISCELLANEOUS TERMS APPLICABLE TO ALL SERVICES

DEFINITION OF APPLE

Depending on your Home Country, “Apple” means:

Apple Inc., located at One Apple Park Way, Cupertino, California, for users in the United States, including Puerto Rico;

Apple Canada Inc., located at 120 Bremner Blvd., Suite 1600, Toronto ON M5J 0A8, Canada for users in Canada;

Apple Services LATAM LLC, located at 1 Alhambra Plaza, Ste 700 Coral Gables, Florida, for users in Mexico, Central or South America, or any Caribbean country or territory (excluding Puerto Rico);

iTunes K.K., located at Roppongi Hills, 6-10-1 Roppongi, Minato-ku, Tokyo 106-6140, Tokyo for users in Japan;

Apple Pty Limited, located at Level 3, 20 Martin Place, Sydney NSW 2000, Australia, for users in Australia or New Zealand, including in any of their territories or affiliated jurisdictions; and

Apple Distribution International Ltd., located at Hollyhill Industrial Estate, Hollyhill, Cork, Republic of Ireland, for all other users.

CONTRACT CHANGES

Apple reserves the right at any time to modify this Agreement and to add new or additional terms or conditions on your use of the Services. Such modifications and additional terms and conditions will be effective immediately and incorporated into this Agreement. Your continued use of the Services will be deemed acceptance thereof.

THIRD-PARTY MATERIALS

Apple is not responsible or liable for third party materials included within or linked from the Content or the Services.

INTELLECTUAL PROPERTY

You agree that the Services, including but not limited to Content, graphics, user interface, audio clips, video clips, editorial content, and the scripts and software used to implement the Services, contain proprietary information and material that is owned by Apple and/or its licensors, and is protected by applicable intellectual property and other laws, including but not limited to copyright. You agree that you will not use such proprietary information or materials in any way whatsoever except for use of the Services for personal, noncommercial uses in compliance with this Agreement. No portion of the Content or Services may be transferred or reproduced in any form or by any means, except as expressly permitted by this Agreement. You agree not to modify, rent, loan, sell, share, or distribute the Services or Content in any manner, and you shall not exploit the Services in any manner not expressly authorized.

The Apple name, the Apple logo, iTunes, iTunes Store, App Store, Apple Books, Apple Music, Apple TV, Apple TV+, Apple Arcade, Apple News, Apple News+, Apple One, Apple Podcasts, Apple Fitness+, and other Apple trademarks, service marks, graphics, and logos used in connection with the Services are trademarks or registered trademarks of Apple in the U.S. and other countries throughout the world. You are granted no right or license with respect to any of the aforesaid trademarks.

COPYRIGHT

Unless otherwise noted, Services and Content provided by Apple are copyrights of Apple Inc. and its subsidiaries.

If you believe that any Content available through the Services infringe a copyright claimed by you, please contact Apple at the following locations:

- Third Party Apps: https://www.apple.com/legal/internet-services/itunes/appstorenotices/

- Apple Books: https://www.apple.com/legal/internet-services/itunes/applebooksnotices/

- Apple TV+ and Apple Fitness+: https://www.apple.com/legal/internet-services/itunes/appletvplusnotices/

- Apple News: https://www.apple.com/legal/internet-services/itunes/applenewsnotices/

- All other Services (including but not limited to iTunes Store, Apple Music, and Apple Podcasts): https://www.apple.com/legal/internet-services/itunes/itunesstorenotices/

TERMINATION AND SUSPENSION OF SERVICES

If you fail, or Apple suspects that you have failed, to comply with any of the provisions of this Agreement, Apple may, without notice to you: (i) terminate this Agreement and/or your Apple ID, and you will remain liable for all amounts due under your Apple ID up to and including the date of termination; and/or (ii) terminate your license to the software; and/or (iii) preclude your access to the Services.

Apple further reserves the right to modify, suspend, or discontinue the Services (or any part or Content thereof) at any time with or without notice to you, and Apple will not be liable to you or to any third party should it exercise such rights.

DISCLAIMER OF WARRANTIES; LIABILITY LIMITATION

APPLE DOES NOT GUARANTEE, REPRESENT, OR WARRANT THAT YOUR USE OF THE SERVICES WILL BE UNINTERRUPTED OR ERROR-FREE, AND YOU AGREE THAT FROM TIME TO TIME APPLE MAY REMOVE THE SERVICES FOR INDEFINITE PERIODS OF TIME, CANCEL THE SERVICES AT ANY TIME, OR OTHERWISE LIMIT OR DISABLE YOUR ACCESS TO THE SERVICES WITHOUT NOTICE TO YOU.

YOU EXPRESSLY AGREE THAT YOUR USE OF, OR INABILITY TO USE, OR ACTIVITY IN CONNECTION WITH THE SERVICES IS AT YOUR SOLE RISK. THE SERVICES AND ALL CONTENT DELIVERED TO YOU THROUGH THE SERVICES ARE (EXCEPT AS EXPRESSLY STATED BY APPLE) PROVIDED "AS IS" AND "AS AVAILABLE" FOR YOUR USE, WITHOUT WARRANTIES OF ANY KIND, EITHER EXPRESS OR IMPLIED, INCLUDING ALL IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, TITLE, AND NONINFRINGEMENT. BECAUSE SOME JURISDICTIONS DO NOT ALLOW THE EXCLUSION OF IMPLIED WARRANTIES, THE ABOVE EXCLUSION OF IMPLIED WARRANTIES MAY NOT APPLY TO YOU.

YOU AGREE THAT UNLESS REQUIRED BY APPLICABLE LAW, APPLE HAS NO RESPONSIBILITY TO CONTINUE MAKING CONTENT AVAILABLE TO YOU THROUGH OUR SERVICES, AND APPLE WILL NOT BE LIABLE TO YOU IF CONTENT, INCLUDING PURCHASED CONTENT, BECOMES UNAVAILABLE FOR DOWNLOAD OR STREAMING.

IN NO CASE SHALL APPLE, ITS DIRECTORS, OFFICERS, EMPLOYEES, AFFILIATES, AGENTS, CONTRACTORS, OR LICENSORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, PUNITIVE, SPECIAL, OR CONSEQUENTIAL DAMAGES ARISING FROM YOUR USE OF ANY OF THE SERVICES OR FOR ANY OTHER CLAIM RELATED IN ANY WAY TO YOUR USE OF THE SERVICES AND/OR CONTENT, INCLUDING, BUT NOT LIMITED TO, ANY ERRORS OR OMISSIONS IN ANY CONTENT, OR ANY INJURY, LOSS OR DAMAGE OF ANY KIND INCURRED AS A RESULT OF THE USE OF ANY CONTENT POSTED, TRANSMITTED, OR OTHERWISE MADE AVAILABLE VIA THE SERVICES, EVEN IF ADVISED OF THEIR POSSIBILITY. BECAUSE SOME COUNTRIES, STATES OR JURISDICTIONS DO NOT ALLOW THE EXCLUSION OR THE LIMITATION OF LIABILITY FOR CONSEQUENTIAL OR INCIDENTAL DAMAGES, IN SUCH COUNTRIES, STATES OR JURISDICTIONS, APPLE'S LIABILITY SHALL BE LIMITED TO THE EXTENT SUCH LIMITATION IS PERMITTED BY LAW.

APPLE SHALL USE REASONABLE EFFORTS TO PROTECT INFORMATION SUBMITTED BY YOU IN CONNECTION WITH THE SERVICES, BUT YOU AGREE THAT YOUR SUBMISSION OF SUCH INFORMATION IS AT YOUR SOLE RISK, AND YOU HEREBY RELEASE APPLE FROM ANY AND ALL LIABILITY TO YOU FOR ANY LOSS OR LIABILITY RELATING TO SUCH INFORMATION IN ANY WAY.

APPLE DOES NOT REPRESENT OR GUARANTEE THAT THE SERVICES WILL BE FREE FROM LOSS, CORRUPTION, ATTACK, VIRUSES, INTERFERENCE, HACKING, OR OTHER SECURITY INTRUSION, AND YOU HEREBY RELEASE APPLE FROM ANY LIABILITY RELATING THERETO. YOU SHALL BE RESPONSIBLE FOR BACKING UP YOUR OWN SYSTEM, INCLUDING ANY CONTENT ACQUIRED OR RENTED THROUGH THE SERVICES.

APPLE IS NOT RESPONSIBLE FOR DATA CHARGES YOU MAY INCUR IN CONNECTION WITH YOUR USE OF THE SERVICES.

WAIVER AND INDEMNITY

BY USING THE SERVICES, YOU AGREE, TO THE EXTENT PERMITTED BY LAW, TO INDEMNIFY AND HOLD APPLE, ITS DIRECTORS, OFFICERS, EMPLOYEES, AFFILIATES, AGENTS, CONTRACTORS, AND LICENSORS HARMLESS WITH RESPECT TO ANY CLAIMS ARISING OUT OF YOUR BREACH OF THIS AGREEMENT, YOUR USE OF THE SERVICES, OR ANY ACTION TAKEN BY APPLE AS PART OF ITS INVESTIGATION OF A SUSPECTED VIOLATION OF THIS AGREEMENT OR AS A RESULT OF ITS FINDING OR DECISION THAT A VIOLATION OF THIS AGREEMENT HAS OCCURRED. YOU AGREE THAT YOU SHALL NOT SUE OR RECOVER ANY DAMAGES FROM APPLE, ITS DIRECTORS, OFFICERS, EMPLOYEES, AFFILIATES, AGENTS, CONTRACTORS, AND LICENSORS AS A RESULT OF ITS DECISION TO REMOVE OR REFUSE TO PROCESS ANY INFORMATION OR CONTENT, TO WARN YOU, TO SUSPEND OR TERMINATE YOUR ACCESS TO THE SERVICES, OR TO TAKE ANY OTHER ACTION DURING THE INVESTIGATION OF A SUSPECTED VIOLATION OR AS A RESULT OF APPLE'S CONCLUSION THAT A VIOLATION OF THIS AGREEMENT HAS OCCURRED. THIS WAIVER AND INDEMNITY PROVISION APPLIES TO ALL VIOLATIONS DESCRIBED IN OR CONTEMPLATED BY THIS AGREEMENT.

STATUTORY EXCEPTIONS FOR PUBLIC INSTITUTIONS

If you are a qualified public educational or government institution and any part of this Agreement, such as, by way of example, all or part of the indemnification section, is invalid or unenforceable against you because of applicable local, national, state or federal law, then that portion shall be deemed invalid or unenforceable, as the case may be, and instead construed in a manner most consistent with applicable governing law.

GOVERNING LAW

Except to the extent expressly provided in the following paragraph, this Agreement and the relationship between you and Apple, and all Transactions on the Services shall be governed by the laws of the State of California, excluding its conflicts of law provisions. You and Apple agree to submit to the personal and exclusive jurisdiction of the courts located within the county of Santa Clara, California, to resolve any dispute or claim arising from this Agreement. If (a) you are not a U.S. citizen; (b) you do not reside in the U.S.; (c) you are not accessing the Service from the U.S.; and (d) you are a citizen of one of the countries identified below, you hereby agree that any dispute or claim arising from this Agreement shall be governed by the applicable law set forth below, without regard to any conflict of law provisions, and you hereby irrevocably submit to the non-exclusive jurisdiction of the courts located in the state, province or country identified below whose law governs:

If you are a citizen of any European Union country or the United Kingdom, Switzerland, Norway or Iceland, the governing law and forum shall be the laws and courts of your usual place of residence.

Specifically excluded from application to this Agreement is that law known as the United Nations Convention on the International Sale of Goods.

OTHER PROVISIONS

This Agreement constitutes the entire agreement between you and Apple and governs your use of the Services, superseding any prior agreements with respect to the same subject matter between you and Apple. You also may be subject to additional terms and conditions that may apply when you use affiliate services, third-party content, third-party software, or additional services such as the Volume Purchase Program. If any part of this Agreement is held invalid or unenforceable, that portion shall be construed in a manner consistent with applicable law to reflect, as nearly as possible, the original intentions of the parties, and the remaining portions shall remain in full force and effect. Apple's failure to enforce any right or provisions in this Agreement will not constitute a waiver of such or any other provision. Apple will not be responsible for failures to fulfill any obligations due to causes beyond its control.

You agree to comply with all local, state, federal, and national laws, statutes, ordinances, and regulations that apply to your use of the Services. Your use of the Services may also be subject to other laws. Risk of loss for all electronically delivered Transactions pass to the acquirer upon electronic transmission to the recipient. No Apple employee or agent has the authority to vary this Agreement.

Apple may notify you with respect to the Services by sending an email message to your email address or a letter via postal mail to your mailing address, or by a posting on the Services. Notices shall become effective immediately. Apple may also contact you by email or push notification to send you additional information about the Services.

You hereby grant Apple the right to take steps Apple believes are reasonably necessary or appropriate to enforce and/or verify compliance with any part of this Agreement. You agree that Apple has the right, without liability to you, to disclose any data and/or information to law enforcement authorities, government officials, and/or a third party, as Apple believes is reasonably necessary or appropriate to enforce and/or verify compliance with any part of this Agreement (including but not limited to Apple's right to cooperate with any legal process relating to your use of the Services and/or Content, and/or a third-party claim that your use of the Services and/or Content is unlawful and/or infringes such third party's rights).

Last Updated: September 20, 2021 """

# **Summarization**

In [9]:
ttt = nltk.tokenize.TextTilingTokenizer()
tokens = ttt.tokenize(text)

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re

In [11]:
def remove_special_characters(text): # used
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [12]:
def paragraph_cleaner(tokens): # used
    tokens_cleaned = []
    for token in tokens:
        token = remove_special_characters(token)
        tokens_cleaned.append(token)
    return tokens_cleaned

In [13]:
tokenized_paragraphs = paragraph_cleaner(tokens)

In [14]:
model_word = Word2Vec(tokenized_paragraphs, min_count=1,sg=1)

In [15]:
def checkNum(s):
    l= ['1','2','3','4','5','6','7','8','9','0']
    check =False

    for i in s:
        if i in l:
            check = True
            break
    if check == True:
        return 1
    else:
        return 0

In [16]:
def meanOfWord(model, paragraph):
    posList=['CD']
    nounList=['NN','NNP','NNS','NNPS']
    value=[]
    count=0
    noun=0
    for word in paragraph:
        a=model_word.similar_by_word(word)
        temp=[]
        for w in a:
            temp.append(w[1])
        posValue=nltk.pos_tag([word])
        wordScore=np.mean(temp)
        if posValue[0][1] in posList:
            count=count+1
        else:
            valueIfNum=checkNum(word)
            count=count+valueIfNum
        if posValue[0][1] in nounList:
            noun=noun + .25
        value.append(wordScore)
    return np.mean(value)+count+noun

In [17]:
score=[]
for index, sentence in enumerate(tokenized_paragraphs):
    i = tokenized_paragraphs.index(sentence)
    meanScore= meanOfWord(model_word,sentence)
    temp = [i,meanScore]
    score.append(temp)

print(score)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  


[[0, 216.19442107532595], [1, 288.9462258032487], [2, 126.69389409982732], [3, 337.69835770868], [4, 221.1935696484459], [5, 194.1916935504377], [6, 388.6913453231829], [7, 96.43900662619856], [8, 315.19451331580575], [9, 230.44164915045755], [10, 33.44599772378908], [11, 332.94660347812305], [12, 40.69117671507227], [13, 147.19134692609595], [14, 155.44522589132598], [15, 322.44781518947735], [16, 108.94512892190302], [17, 91.44736397643322], [18, 90.94153781105946], [19, 246.9491434842516], [20, 145.69395542074204], [21, 152.44269508870016], [22, 58.20073864404264], [23, 231.4430696227598], [24, 308.4398252113327], [25, 175.94534945549964], [26, 128.9386960542415], [27, 196.69180209396202], [28, 324.69584966458683], [29, 129.69769229953238], [30, 45.69629171435079], [31, 331.9482875866585], [32, 234.1939681740064], [33, 253.94443519268137], [34, 195.69718956248366], [35, 199.6912544586025], [36, 218.94436169935653], [37, 78.19500909288959], [38, 257.1945818558999], [39, 391.445287535

In [18]:
token_scores = []
for scores in score:
  token_scores.append(scores[1])
sorted_tokens = list(zip(tokens,token_scores))
sorted_tokens.sort(key= lambda a:a[1], reverse=1)
sorted_tokens

[('\n\nApple Pty Limited, located at Level 3, 20 Martin Place, Sydney NSW 2000, Australia, for users in Australia or New Zealand, including in any of their territories or affiliated jurisdictions; and\n\nApple Distribution International Ltd., located at Hollyhill Industrial Estate, Hollyhill, Cork, Republic of Ireland, for all other users.\n\nCONTRACT CHANGES\n\nApple reserves the right at any time to modify this Agreement and to add new or additional terms or conditions on your use of the Services. Such modifications and additional terms and conditions will be effective immediately and incorporated into this Agreement. Your continued use of the Services will be deemed acceptance thereof.\n\nTHIRD-PARTY MATERIALS\n\nApple is not responsible or liable for third party materials included within or linked from the Content or the Services.\n\nINTELLECTUAL PROPERTY\n\nYou agree that the Services, including but not limited to Content, graphics, user interface, audio clips, video clips, editor

In [19]:
def relevant_text_ranker(n,sorted_tokens):
  relevant_text = []
  if(n==1):
    for i in range(5):
      relevant_text.append(sorted_tokens[i][0])
  if(n==2):
    for i in range(20):
      relevant_text.append(sorted_tokens[i][0])
  if(n==3):
    points = int(input())
    for i in range(points):
      relevant_text.append(sorted_tokens[i][0])
    return points,relevant_text
  return relevant_text
    

In [20]:
relevant_text_free = relevant_text_ranker(1,sorted_tokens)
relevant_text_premium = relevant_text_ranker(2,sorted_tokens)
points, relevant_text_business = relevant_text_ranker(3,sorted_tokens)

15


In [21]:
relevant_text_free

['\n\nApple Pty Limited, located at Level 3, 20 Martin Place, Sydney NSW 2000, Australia, for users in Australia or New Zealand, including in any of their territories or affiliated jurisdictions; and\n\nApple Distribution International Ltd., located at Hollyhill Industrial Estate, Hollyhill, Cork, Republic of Ireland, for all other users.\n\nCONTRACT CHANGES\n\nApple reserves the right at any time to modify this Agreement and to add new or additional terms or conditions on your use of the Services. Such modifications and additional terms and conditions will be effective immediately and incorporated into this Agreement. Your continued use of the Services will be deemed acceptance thereof.\n\nTHIRD-PARTY MATERIALS\n\nApple is not responsible or liable for third party materials included within or linked from the Content or the Services.\n\nINTELLECTUAL PROPERTY\n\nYou agree that the Services, including but not limited to Content, graphics, user interface, audio clips, video clips, editori

In [22]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
nlp= spacy.load("en_core_web_sm")
from heapq import nlargest
!pip install transformers
import transformers
from transformers import pipeline

In [23]:
def text_summarizer_free(text):
  summarizer = pipeline("summarization")
  summarized = summarizer(text[:4900], min_length=50, max_length=150)
  summ=' '.join([str(i) for i in summarized])
  summ=summ.replace("{","")
  summ=summ.replace("''","")
  return summ[18:-2]
def text_summarizer_premium(text):
  summarizer = pipeline("summarization")
  summarized = summarizer(text[:4900], min_length=50, max_length=150)
  summ=' '.join([str(i) for i in summarized])
  summ=summ.replace("{","")
  summ=summ.replace("''","")
  return summ[18:-2]
def text_summarizer_business(text):
  summarizer = pipeline("summarization")  
  summarized = summarizer(text[:4900], min_length=50, max_length=150)
  summ=' '.join([str(i) for i in summarized])
  summ=summ.replace("{","")
  summ=summ.replace("''","")
  return summ[18:-2]
   

In [ ]:
summarized_text_free = text_summarizer_free(relevant_text_free)
summarized_text_premium = text_summarizer_premium(relevant_text_premium)
summarized_text_business = text_summarizer_business(relevant_text_business)
print(summarized_text_free)
print(summarized_text_premium)
print(summarized_text_business)

# **Training our CWI Model**

In [24]:
Dataset = namedtuple('Dataset', 'name, train, test')
Model = namedtuple('Model', 'type, name, dimension, corpus, model')

In [25]:
!wget https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/complex-word-identification-dataset/cwishareddataset.zip

--2022-04-30 16:51:15--  https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/complex-word-identification-dataset/cwishareddataset.zip
Resolving www.inf.uni-hamburg.de (www.inf.uni-hamburg.de)... 134.100.36.5
Connecting to www.inf.uni-hamburg.de (www.inf.uni-hamburg.de)|134.100.36.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1075894 (1.0M) [application/zip]
Saving to: ‘cwishareddataset.zip’

cwishareddataset.zi 100%[===================>]   1.03M   440KB/s    in 2.4s    

2022-04-30 16:51:19 (440 KB/s) - ‘cwishareddataset.zip’ saved [1075894/1075894]



In [26]:
!unzip cwishareddataset.zip

Archive:  cwishareddataset.zip
   creating: traindevset/
   creating: traindevset/german/
  inflating: traindevset/german/German_Train.tsv  
  inflating: traindevset/german/German_Dev.tsv  
  inflating: traindevset/README.md   
   creating: traindevset/english/
  inflating: traindevset/english/WikiNews_Train.tsv  
  inflating: traindevset/english/Wikipedia_Train.tsv  
  inflating: traindevset/english/Wikipedia_Dev.tsv  
  inflating: traindevset/english/News_Dev.tsv  
  inflating: traindevset/english/News_Train.tsv  
  inflating: traindevset/english/WikiNews_Dev.tsv  
   creating: traindevset/spanish/
  inflating: traindevset/spanish/Spanish_Train.tsv  
  inflating: traindevset/spanish/Spanish_Dev.tsv  
  inflating: README.md               
   creating: testset/
   creating: testset/german/
  inflating: testset/german/German_Test.tsv  
  inflating: testset/README.md       
   creating: testset/english/
  inflating: testset/english/Wikipedia_Test.tsv  
   creating: __MACOSX/
   creating:

In [27]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
MAIN_PATH_DATASET = "traindevset/english/"
genres = ['Wikipedia', 'WikiNews', 'News']
datasets = ['Train', 'Dev']
columns = ['id', 'sentence', "start", "end", "target", 
           "nat", "non_nat", "nat_marked", "non_nat_marked", "binary", "prob"]


datasets = [Dataset('Wikipedia', 'Train', 'Dev'),
            Dataset('WikiNews', 'Train', 'Dev'),
            Dataset('News', 'Train', 'Dev')]

feature_categories = []

def load_df(path):
    df = pd.read_csv(path, header=None, sep = "\t")
    df.columns = columns
    return df

datasets = [Dataset(d.name, load_df(MAIN_PATH_DATASET + d.name + '_' + d.train + '.tsv'),
                            load_df(MAIN_PATH_DATASET + d.name + '_' + d.test + '.tsv'))
                            for d in datasets]

In [28]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-04-30 16:51:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-30 16:51:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-30 16:51:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [29]:
!unzip glove.6B.zip -d embeddings

Archive:  glove.6B.zip
  inflating: embeddings/glove.6B.50d.txt  
  inflating: embeddings/glove.6B.100d.txt  
  inflating: embeddings/glove.6B.200d.txt  
  inflating: embeddings/glove.6B.300d.txt  


In [30]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

MAIN_PATH = 'embeddings/'


glove_models = []

glove_defs = [ Model('glove', 'glove.6B.300d.txt', 300, 'wikipedia+gigaword5', None)]
              
for model in glove_defs:
    glove_file = MAIN_PATH + model.name
    tmp_file = get_tmpfile(model.name + '-temp')
    glove2word2vec(glove_file, tmp_file)
    vecs = KeyedVectors.load_word2vec_format(tmp_file)
    glove_models.append(Model(model.type, model.name, model.dimension, model.corpus, vecs))
    print('load model : {}'.format(model.name))
    
print(glove_models)

load model : glove.6B.300d.txt
[Model(type='glove', name='glove.6B.300d.txt', dimension=300, corpus='wikipedia+gigaword5', model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7ff870496d90>)]


In [31]:
dataframe = datasets[0].train[0:30]

def overlaps(start1, end1, start2, end2):
    return bool(range(max(start1, start2), min(end1, end2)+1))

def extract_ngrams_group(group):
    targets = zip(group['target'].values.tolist(), group['start'].values.tolist(),
                 group['end'].values.tolist(), group['binary'].values.tolist())
    for word, start, end, binary in targets:
        tokens = word.split()
        if len(tokens)>1:
            olap_words = [(w, b) for w, s, e, b in targets if overlaps(start, end, s, e)]
            
    
grouped = dataframe.groupby('sentence').apply(lambda group : extract_ngrams_group(group))

In [32]:
wordlist_lowercased = set(i.lower() for i in brown.words())
print (len(wordlist_lowercased))

49815


In [33]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))

def remove_punctuation(text):
    return text.translate(tbl)

@lru_cache(maxsize=None)
def all_tokens_with_index(context):
    curr_pos = 0
    targets = []
    j = 0
    w = 0
    curr_split = ''
    ctx_split = context.split()
    whitespaces = re.findall('\s+', context)
    num_whitespaces = [len(token) for token in whitespaces]
    num_whitespaces.append(1)
    tokens = word_tokenize(context)
    tokens = ['"' if token not in context else token for token in tokens]
    for index, token in enumerate(tokens, 1):
        targets.append((token, index, curr_pos, (curr_pos + len(token))))
        curr_pos += len(token)
        curr_split += token
        if ctx_split[j] == curr_split:
            curr_pos += num_whitespaces[w]
            j += 1
            w += 1
            curr_split = ''
    return [val for val in targets if val[0] != '"']

def build_vocabulary(sentences, embedding_model, dimension):
    all_words = [tpl[0] for sentence in sentences for tpl in sentence['seq']] + list(wordlist_lowercased)
    print('# Words : {}'.format(len(all_words)))
    counter = Counter(all_words)
    vocab_size = len(counter) + 1
    print('# Vocab : {}'.format(vocab_size))
    print('# embeding model  : {}'.format(len(embedding_model.vocab)))   
    word2index = {word : index for index, (word, count) in enumerate(counter.most_common(), 1)}
    index2word = {index : word for word, index in word2index.items()}
    # +1 required for pad token
    embedding_matrix = np.zeros(((vocab_size), dimension))
    missing_embed_words = []
    i_ = 0
    for word, index in word2index.items():
        if word in embedding_model.vocab:
            embedding = embedding_model[word]
        else:
             i_ +=1
             continue
        embedding_matrix[index] = embedding
    missing_embed_count = len(missing_embed_words)
    print('# Words missing embedding : {}'.format(missing_embed_count))
    print('Embedding shape : {}'.format(embedding_matrix.shape))
    print("i: ", i_ )
    return word2index, index2word, embedding_matrix

def forward_transformation(dataframe, lowercase = True, filter_punc = True, filtering = "a132"):
    grouped = dataframe.groupby('sentence').apply(lambda row : 
                        {'sent_id' : list(set(row['sent_id']))[0],
                         'sentence' : list(set(row['sentence']))[0], 
                         'tags': [tag for tag in zip(row['target'], 
                            row['start'], row['end'], row['binary'], row['prob'])]})
    sentences = []
    for vals in grouped:
        sent_id = vals['sent_id']
        sentence = vals['sentence']
        tags = vals['tags']
        tags_without_labels = [(word, start, end) for word, start, end, binary, prob in tags]
        all_tokens = all_tokens_with_index(sentence)
        sent_repr = [(word, start, end, tags[tags_without_labels.index((word, start, end))][3],
                     tags[tags_without_labels.index((word, start, end))][4])
           if (word, start, end) in tags_without_labels 
          else (word, start, end, 0, 0.0) for word, index, start, end in all_tokens]
        if lowercase:
            sent_repr = [(word.lower(), start, end, binary, prob) 
                         for word, start, end, binary, prob in sent_repr]
        if filter_punc:
            sent_repr = list(filter(lambda vals : remove_punctuation(vals[0]), sent_repr))
        if filtering:
            sent_repr = list(filter(lambda vals : vals[0] != "'s", sent_repr))
            sent_repr = list(filter(lambda vals : vals[0] != "``", sent_repr))
        sentences.append({'sent_id' : sent_id, 'sentence' : sentence, 'seq' : sent_repr})
    return sentences

def split_sentence_seqs(sentences):
    words, start_end, binary, prob = [], [], [] ,[]
    for sent in sentences:
        sequence = sent['seq']
        curr_w, curr_se, curr_b, curr_p = map(list, zip(*[(vals[0], 
            (vals[1], vals[2]), vals[3], vals[4]) for vals in sequence]))
        words.append(curr_w)
        start_end.append(curr_se)
        binary.append(curr_b)
        prob.append(curr_p)
    return words, start_end, binary, prob

In [34]:
datasets.append(Dataset('train_all_test_wiki', 
        datasets[0].train.append(datasets[1].train).append(datasets[2].train), datasets[0].test))

In [35]:
# Append train and test set
dataset_sel = datasets[3]
train_num_rows = dataset_sel.train.shape[0]
train_num_sents = len(list(set(dataset_sel.train.sentence.values.tolist())))

test_num_rows = dataset_sel.test.shape[0]
test_num_sents = len(list(set(dataset_sel.test.sentence.values.tolist())))

dataset = dataset_sel.train.append(dataset_sel.test)
dataset['sent_id'] = dataset.groupby('sentence').ngroup()
dataset_num_rows = dataset.shape[0]
dataset_num_sents = len(list(set(dataset.sentence.values.tolist())))

print('# Rows train : {}'.format(train_num_rows))
print('# Rows test : {}'.format(test_num_rows))
print('# Rows dataset : {}'.format(dataset_num_rows))

print('# Sents train : {}'.format(train_num_sents))
print('# Sents test : {}'.format(test_num_sents))
print('# Sents dataset : {}'.format(dataset_num_sents))

# Rows train : 27299
# Rows test : 694
# Rows dataset : 27993
# Sents train : 1988
# Sents test : 53
# Sents dataset : 2041


In [36]:
sentences = forward_transformation(dataset)
train_sentences = sentences[:train_num_sents]
test_sentences = sentences[train_num_sents:]

In [37]:
words, start_end, binary, prob = split_sentence_seqs(sentences)

In [38]:
sentence_lens = [len(sent) for sent in words]

In [39]:
embedding_model = glove_models[0].model
dimension = embedding_model.vector_size
word2index, index2word, embedding = build_vocabulary(sentences, embedding_model, dimension)

# Words : 96437
# Vocab : 52455
# embeding model  : 400000
# Words missing embedding : 0
Embedding shape : (52455, 300)
i:  9834


In [40]:

words_with_indices = [[word2index[word] for word in sent] for sent in words]
sent_lens = [len(sentence['seq']) for sentence in sentences]
sent_max_length = np.max(sent_lens)
print('Max length sentence : {}'.format(sent_max_length))

from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
words_padded = pad_sequences(maxlen=sent_max_length, sequences=words_with_indices, padding="post", value=0)
binary_padded = pad_sequences(maxlen=sent_max_length, sequences=binary, padding="post", value=0)
prob_padded = pad_sequences(maxlen=sent_max_length, sequences=prob, padding="post", value=0, dtype="float")

binary_padded_categorical = [to_categorical(clazz, num_classes=2) for clazz in binary_padded]

Max length sentence : 103


In [41]:

# (1) Training set
train_words_padded = words_padded[:train_num_sents]
train_binary_padded = binary_padded[:train_num_sents]
train_binary_padded_categorical = binary_padded_categorical[:train_num_sents]
train_prob_padded = prob_padded[:train_num_sents]
train_start_end = start_end[:train_num_sents]

# (2) Test set
test_words_padded = words_padded[train_num_sents:]
test_binary_padded = binary_padded[train_num_sents:]
test_binary_padded_categorical = binary_padded_categorical[train_num_sents:]
test_prob_padded = prob_padded[train_num_sents:]
test_start_end = start_end[train_num_sents:]

print('Training set length : {}'.format(len(train_words_padded)))
print('Test set length : {}'.format(len(test_words_padded)))

Training set length : 1988
Test set length : 53


In [42]:
from sklearn.metrics import f1_score
import keras.callbacks
from keras import backend as K
class Metrics(keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.f1_scores = []
        self.validation_data = validation_data
        
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0]))
        targ = self.validation_data[1]
        targ = np.array(targ)
        shape = targ.shape
        targ = targ.reshape((shape[0]*shape[1], shape[2]))
        targ = np.argmax(targ, axis = 1)
        predict = predict.reshape((shape[0]*shape[1]), shape[2])
        predict = np.argmax(predict, axis = 1)
        self.f1s=f1_score(targ, predict)
        print("\nF1 Score:")
        print(f1_score(targ, np.ones(shape[0]*shape[1])))
        self.f1_scores.append(self.f1s)
        return

In [43]:
from keras.models import Model, Input
from keras import backend as K
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

vocab_size = embedding.shape[0]
dimension = embedding.shape[1]

np.set_printoptions(threshold=np.inf)

in_seq = Input(shape=(sent_max_length,))
embed = Embedding(input_dim=vocab_size, output_dim=dimension, \
                  weights=[embedding], input_length=sent_max_length)(in_seq)
drop = Dropout(0.1)(embed)
lstm = Bidirectional(LSTM(units=150, return_sequences=True, recurrent_dropout=0.1))(drop)
out = TimeDistributed(Dense(2, activation="softmax"))(lstm) 

model = Model(in_seq, out)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

metrics = Metrics((test_words_padded, np.array(test_binary_padded_categorical)))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 103)]             0         
                                                                 
 embedding (Embedding)       (None, 103, 300)          15736500  
                                                                 
 dropout (Dropout)           (None, 103, 300)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 103, 300)         541200    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 103, 2)           602       
 ibuted)                                                         
                                                                 
Total params: 16,278,302
Trainable params: 16,278,302
Non-tra

In [ ]:
history = model.fit(train_words_padded, np.array(train_binary_padded_categorical), batch_size=10, 
                    epochs=3, validation_data = (test_words_padded, np.array(test_binary_padded_categorical)), 
                    verbose=1, callbacks=[metrics])

Epoch 1/3
199/199 [==============================] - ETA: 0s - loss: 0.0849 - accuracy: 0.9680
F1 Score:
0.05660377358490566
199/199 [==============================] - 187s 905ms/step - loss: 0.0849 - accuracy: 0.9680 - val_loss: 0.0492 - val_accuracy: 0.9786
Epoch 2/3
199/199 [==============================] - ETA: 0s - loss: 0.0401 - accuracy: 0.9829
F1 Score:
0.05660377358490566
199/199 [==============================] - 181s 910ms/step - loss: 0.0401 - accuracy: 0.9829 - val_loss: 0.0432 - val_accuracy: 0.9809
Epoch 3/3
199/199 [==============================] - ETA: 0s - loss: 0.0297 - accuracy: 0.9877
F1 Score:
0.05660377358490566
199/199 [==============================] - 180s 903ms/step - loss: 0.0297 - accuracy: 0.9877 - val_loss: 0.0452 - val_accuracy: 0.9820


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
epoch_f1s = plt.plot(metrics.f1_scores)
plt.show()

# **Checking Our Simplification Output**

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [45]:
model_save_name = 'model_CWI_full.h5'
path_dir = F"/content/gdrive/My Drive/Models/{model_save_name}"

In [46]:
from keras.models import load_model

model_cwi = load_model(path_dir)

In [47]:
stop_words_ = set(stopwords.words('english'))
def cleaner(word):
  #Remove links
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                '', word, flags=re.MULTILINE)
  word = re.sub('[\W]', ' ', word)
  word = re.sub('[^a-zA-Z]', ' ', word)
  return word.lower().strip()

In [48]:

def process_input(input_text):
  input_text = cleaner(input_text)
  clean_text = []
  index_list =[]
  input_token = []
  index_list_zipf = []
  for i, word in enumerate(input_text.split()):
    if word in word2index:
      clean_text.append(word)
      input_token.append(word2index[word])
    else:
      index_list.append(i)
  input_padded = pad_sequences(maxlen=sent_max_length, sequences=[input_token], padding="post", value=0)
  return input_padded, index_list, len(clean_text)


In [49]:
def complete_missing_word(pred_binary, index_list, len_list):
  list_cwi_predictions = list(pred_binary[0][:len_list])
  for i in index_list:
    list_cwi_predictions.insert(i, 0)
  return list_cwi_predictions


In [2]:
!pip install transformers
!pip install torch==1.11.0

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 30.8 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [62]:
import torch
from transformers import AutoModelForMaskedLM, LongformerTokenizer, LongformerForMaskedLM
from transformers import BertTokenizer, BertModel, BertForMaskedLM

longformer_model = 'allenai/longformer-base-4096'
bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [3]:
!pip install wordfreq

     |████████████████████████████████| 56.8 MB 1.3 MB/s 
     |████████████████████████████████| 749 kB 60.8 MB/s 
     |████████████████████████████████| 181 kB 51.1 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20


In [4]:
!pip install nltk==3.6.4


     |████████████████████████████████| 1.5 MB 4.3 MB/s 
Reason for being yanked: More recent version of regex library required (https://github.com/nltk/nltk/issues/2833)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [50]:
from wordfreq import zipf_frequency
zipf_frequency('cancel', 'en')


4.18

In [51]:
from wordfreq import zipf_frequency
zipf_frequency('thwart', 'en')

3.06

In [52]:
zipf_frequency("compatible",'en')

3.97

In [ ]:
sample_text = """Germany (German: Deutschland, German pronunciation: [ˈdɔʏtʃlant]), officially the Federal Republic of Germany,[e] is a country at the intersection of Central and Western Europe. It is situated between the Baltic and North seas to the north, and the Alps to the south; covering an area of 357,022 square kilometres (137,847 sq mi), with a population of over 83 million within its 16 constituent states. It borders Denmark to the north, Poland and the Czech Republic to the east, Austria and Switzerland to the south, and France, Luxembourg, Belgium, and the Netherlands to the west. Germany is the second-most populous country in Europe after Russia, as well as the most populous member state of the European Union. Its capital and largest city is Berlin, and its financial centre is Frankfurt; the largest urban area is the Ruhr.Various Germanic tribes have inhabited the northern parts of modern Germany since classical antiquity. A region named Germania was documented before AD 100. In the 10th century, German territories formed a central part of the Holy Roman Empire. During the 16th century, northern German regions became the centre of the Protestant Reformation. Following the Napoleonic Wars and the dissolution of the Holy Roman Empire in 1806, the German Confederation was formed in 1815. In 1871, Germany became a nation-state when most of the German states unified into the Prussian-dominated German Empire. After World War I and the German Revolution of 1918–1919, the Empire was replaced by the semi-presidential Weimar Republic. The Nazi seizure of power in 1933 led to the establishment of a dictatorship, World War II, and the Holocaust. After the end of World War II in Europe and a period of Allied occupation, Germany was divided into the Federal Republic of Germany, generally known as West Germany, and the German Democratic Republic, East Germany. The Federal Republic of Germany was a founding member of the European Economic Community and the European Union, while the German Democratic Republic was a communist Eastern Bloc state and member of the Warsaw Pact. After the fall of communism, German reunification saw the former East German states join the Federal Republic of Germany on 3 October 1990—becoming a federal parliamentary republic led by a chancellor.Germany is a great power with a strong economy; it has the largest economy in Europe, the world's fourth-largest economy by nominal GDP, and the fifth-largest by PPP. As a global leader in several industrial, scientific and technological sectors, it is both the world's third-largest exporter and importer of goods. As a developed country, which ranks very high on the Human Development Index, it offers social security and a universal health care system, environmental protections, and a tuition-free university education. Germany is also a member of the United Nations, NATO, the G7, the G20, and the OECD. It also has the fourth-greatest number of UNESCO World Heritage Sites."""
tokenizer(sample_text,add_prefix_space=True)

Keyword arguments {'add_prefix_space': True} not recognized.
Token indices sequence length is longer than the specified maximum sequence length for this model (591 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101, 2762, 1006, 2446, 1024, 28668, 1010, 2446, 15498, 1024, 1031, 100, 1033, 1007, 1010, 3985, 1996, 2976, 3072, 1997, 2762, 1010, 1031, 1041, 1033, 2003, 1037, 2406, 2012, 1996, 6840, 1997, 2430, 1998, 2530, 2885, 1012, 2009, 2003, 4350, 2090, 1996, 11275, 1998, 2167, 11915, 2000, 1996, 2167, 1010, 1998, 1996, 13698, 2000, 1996, 2148, 1025, 5266, 2019, 2181, 1997, 26231, 1010, 6185, 2475, 2675, 3717, 1006, 14989, 1010, 6391, 2581, 5490, 2771, 1007, 1010, 2007, 1037, 2313, 1997, 2058, 6640, 2454, 2306, 2049, 2385, 13794, 2163, 1012, 2009, 6645, 5842, 2000, 1996, 2167, 1010, 3735, 1998, 1996, 5569, 3072, 2000, 1996, 2264, 1010, 5118, 1998, 5288, 2000, 1996, 2148, 1010, 1998, 2605, 1010, 10765, 1010, 5706, 1010, 1998, 1996, 4549, 2000, 1996, 2225, 1012, 2762, 2003, 1996, 2117, 1011, 2087, 20151, 2406, 1999, 2885, 2044, 3607, 1010, 2004, 2092, 2004, 1996, 2087, 20151, 2266, 2110, 1997, 1996, 2647, 2586, 1012, 2049, 3007, 1998, 2922, 2103, 2003, 4068, 1010, 1998, 2049, 3361

In [53]:
def get_bert_candidates(input_text, list_cwi_predictions, numb_predictions_displayed = 10):
  list_candidates_bert = []
  for word,pred  in zip(input_text.split(), list_cwi_predictions):
    if (pred and (pos_tag([word])[0][1] in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD' ]))  or (zipf_frequency(word, 'en')) <3.5:
      replace_word_mask = input_text.replace(word, '[MASK]')
      text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
      tokenized_text = tokenizer.tokenize(text)
      masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      segments_ids = [0]*len(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])
      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor, token_type_ids=segments_tensors)
          predictions = outputs[0][0][masked_index]
      predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
      predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
      list_candidates_bert.append((word, predicted_tokens))
  return list_candidates_bert
  



In [ ]:
list_texts = [ 
 'The Risk That Students Could Arrive at School With the Coronavirus As schools grapple with how to reopen, new estimates show that large parts of the country would probably see infected students if classrooms opened now.',
 'How a photograph of a young man cradling his dying friend sent me on a journey across India.',
 'Pro-democracy parties, which had hoped to ride widespread discontent to big gains, saw the yearlong delay as an attempt to thwart them.',
 'Night after night, calm gave way to chaos. See what happened between the protesters and the federal agents.',
 'Contact Tracing Is Failing in Many States. Here is Why. Inadequate testing and protracted delays in producing results have crippled tracking and hampered efforts to contain major outbreaks.',
 'After an initial decrease in the youth detention population, the rate of release has slowed, and the gap between white youth and Black youth has grown.'
 'A laboratory experiment hints at some of the ways the virus might elude antibody treatments. Combining therapies could help, experts said.',
 'Though I may not be here with you, I urge you to answer the highest calling of your heart and stand up for what you truly believe.',
 'The research does not prove that infected children are contagious, but it should influence the debate about reopening schools, some experts said.',
 'Dropping antibody counts are not a sign that our immune system is failing against the coronavirus, nor an omen that we can not develop a viable vaccine.',
 'The Senate majority leader has said he will not approve a stimulus package without a “liability shield,” but top White House officials say they do not see it as essential.',
 'Campaign efforts to refocus come as the president continues to push divisive messages that have frustrated his own party.'
] 
list_texts

['The Risk That Students Could Arrive at School With the Coronavirus As schools grapple with how to reopen, new estimates show that large parts of the country would probably see infected students if classrooms opened now.',
 'How a photograph of a young man cradling his dying friend sent me on a journey across India.',
 'Pro-democracy parties, which had hoped to ride widespread discontent to big gains, saw the yearlong delay as an attempt to thwart them.',
 'Night after night, calm gave way to chaos. See what happened between the protesters and the federal agents.',
 'Contact Tracing Is Failing in Many States. Here is Why. Inadequate testing and protracted delays in producing results have crippled tracking and hampered efforts to contain major outbreaks.',
 'After an initial decrease in the youth detention population, the rate of release has slowed, and the gap between white youth and Black youth has grown.A laboratory experiment hints at some of the ways the virus might elude antibody

In [ ]:
free_summary = [' This Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple Podcasts, Apple TV Channels, Game Center, iTunes . To use our Services, you need compatible hardware, software (latest version recommended and sometimes required) and Internet access (fees may apply) Apple may suspend or cancel payment or refuse a refund request if we find evidence of fraud, abuse or unlawful or other manipulative behavior.'
                 ]
premium_summary = [ ' This Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple Podcasts, Apple TV Channels, Game Center, iTunes . To use our Services, you need compatible hardware, software (latest version recommended and sometimes required) and Internet access (fees may apply) Apple may suspend or cancel payment or refuse a refund request if we find evidence of fraud, abuse, or unlawful or other manipulative behavior that entitles to a corresponding behavior .']
business_summary = ["This Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple Podcasts, Apple TV Channels, Game Center, iTunes . To use our Services, you need compatible hardware, software (latest version recommended and sometimes required) and Internet access (fees may apply) Apple may suspend or cancel payment or refuse a refund request if we find evidence of fraud, abuse, or unlawful or other manipulative behavior that entitles to a corresponding behavior . An Apple ID is the account you use across the ecosystem . Use of Game Center is subject to this Agreement and also requires a Game Center . You can acquire Content on our Services for free or for a charge, either of which is referred to as a “Transactions” Each Transaction is an electronic contract between you and Apple, and/or you and the entity providing the content providing the Content . If you pre-order Content, you will be charged when the Content is delivered to you (unless you cancel prior to the Content's availability). Apple is not responsible for any losses arising from the unauthorized use of your account . You must be age 13 (or equivalent in your minimum in your age in your home country of use of our Services) You are responsible for the use of this Agreement. You can use your Apple ID, and you agree that your account has been compromised."]

In [54]:
def listToString(s): 
    str1 = " " 
    return (str1.join(s))

In [55]:
def listToString2(s): 
    str1 = "" 
    return (str1.join(s))

In [ ]:
def summary_processor(summary):
  summary_1 = []
  summary_2 = []
  summary_3 = []
  summary_4 = []
  summary_5 = []
  if(len(summary.split())>328):
    split_summary_1 = summary.split(maxsplit=82)[:82]
    summary_1.append(listToString(split_summary_1))
    summary = summary.replace(listToString(split_summary_1),'')
    split_summary_2 = summary.split(maxsplit=82)[:82]
    summary_2.append(listToString(split_summary_2))
    summary = summary.replace(listToString(split_summary_2),'')
    split_summary_3 = summary.split(maxsplit=82)[:82]
    summary_3.append(listToString(split_summary_3))
    summary = summary.replace(listToString(split_summary_3),'')
    split_summary_4 = summary.split(maxsplit=82)[:82]
    summary_4.append(listToString(split_summary_4))
    summary = summary.replace(listToString(split_summary_4),'')
    split_summary_5 = summary.split(maxsplit=82)
    summary_5.append(listToString(split_summary_5))
    return summary_1,summary_2, summary_3, summary_4, summary_5
  if(len(summary.split())>246):
    split_summary_1 = summary.split(maxsplit=82)[:82]
    summary_1.append(listToString(split_summary_1))
    summary = summary.replace(listToString(split_summary_1),'')
    split_summary_2 = summary.split(maxsplit=82)[:82]
    summary_2.append(listToString(split_summary_2))
    summary = summary.replace(listToString(split_summary_2),'')
    split_summary_3 = summary.split(maxsplit=82)[:82]
    summary_3.append(listToString(split_summary_3))
    summary = summary.replace(listToString(split_summary_3),'')
    split_summary_4 = summary.split(maxsplit=82)
    summary_4.append(listToString(split_summary_4))
    return summary_1,summary_2, summary_3, summary_4
  if(len(summary.split())>164):
    split_summary_1 = summary.split(maxsplit=82)[:82]
    summary_1.append(listToString(split_summary_1))
    summary = summary.replace(listToString(split_summary_1),'')
    split_summary_2 = summary.split(maxsplit=82)[:82]
    summary_2.append(listToString(split_summary_2))
    summary = summary.replace(listToString(split_summary_2),'')
    split_summary_3 = summary.split(maxsplit=82)
    summary_3.append(listToString(split_summary_3))
    return summary_1,summary_2, summary_3
  if(len(summary.split())>82):
    split_summary_1 = summary.split(maxsplit=82)[:82]
    summary_1.append(listToString(split_summary_1))
    summary = summary.replace(listToString(split_summary_1),'')
    split_summary_2 = summary.split(maxsplit=82)
    summary_2.append(listToString(split_summary_2))
    return summary_1,summary_2
  else:
    return summary
summary_list = []
summary_list.append(summarized_text_business)
processed_summary = summary_processor(summary_list[0])
print(processed_summary)

NameError: ignored

In [ ]:
print(list(processed_summary))

[['This Agreement governs your use of Apple’s Services (“Services” – e.g., and where available, App Store, Apple Arcade, Apple Books, Apple Fitness+, Apple Music, Apple News, Apple Podcasts, Apple TV Channels, Game Center, iTunes . To use our Services, you need compatible hardware, software (latest version recommended and sometimes required) and Internet access (fees may apply) Apple may suspend or cancel payment or refuse a refund request if we find evidence of fraud, abuse, or unlawful or other manipulative behavior that entitles'], ['to a corresponding behavior . An Apple ID is the account you use across the ecosystem . Use of Game Center is subject to this Agreement and also requires a Game Center . You can acquire Content on our Services for free or for a charge, either of which is referred to as a “Transactions” Each Transaction is an electronic contract between you and Apple, and/or you and the entity providing the content providing the Content . If you pre-order Content, you'],

In [ ]:
Original_Text = ''
Simplified_Text = ''
iteration = 0
for input_text in relevant_text_free:
  new_text = input_text[:1170]
  input_padded, index_list, len_list = process_input(input_text[:1170])
  pred_cwi = model_cwi.predict(input_padded)
  pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
  complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
  bert_candidates =   get_bert_candidates(input_text[:1170], complete_cwi_predictions)
  for word_to_replace, l_candidates in bert_candidates:
    tuples_word_zipf = []
    for w in l_candidates:
        tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
    tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
    new_text = re.sub(re.escape(word_to_replace), tuples_word_zipf[0][0], new_text) 
  new_text = re.sub("\n\n","\n",new_text)
  
  Original_Text += input_text[:1170]
  Simplified_Text += new_text

print("Original text: ", Original_Text) 
print("Simplified text:", Simplified_Text, "\n")

NameError: ignored

# **Grammatical Correctness**

In [56]:
!pip install language_tool_python
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')
sample_text = 'This Agreement rules your use of the Services (“Services” and e.g., and where available, App store Apple and Apple Books, Apple Fitness+, Apple Music, Apple News, Apple and Apple TV Channels, game Center, iTunes . To use our Services, you need compatible computer software (latest development recommended and sometimes required) and Internet access costs may amount Apple may stop or delay money or make a refund request if we find cause of data use or criminal or other criminal practice that .to a corresponding practice and An Apple ID is the account you use across the community and Use of Game Center is subject to this Agreement and also requires a Game Center and You can acquire Content on our Services for free or for a charge, either of which is referred to as a “Transactions” Each Transaction is an electronic contract between you and Apple, and/or you and the entity providing the content providing the Content and If you pre-order Content, youwill be charged when the Content is delivered to you (unless you check prior to the full availability)the Apple is not responsible for any losses suffering from the illegal use of your account the You must be age 13 and equivalent in your minimum in your age in your home country of use of our Services) You is responsible for the use of this Agreementthe You can use your Apple ID, and you agree that your account has been compromised'
def grammar_corrector(text):
  matches = tool.check(text)
  my_mistakes = []
  my_corrections = []
  start_positions = []
  end_positions = []
  
  for rules in matches:
      if len(rules.replacements)>0:
          start_positions.append(rules.offset)
          end_positions.append(rules.errorLength+rules.offset)
          my_mistakes.append(text[rules.offset:rules.errorLength+rules.offset])
          my_corrections.append(rules.replacements[0])
      
  
      
  my_new_text = list(text)
  
  
  for m in range(len(start_positions)):
      for i in range(len(text)):
          my_new_text[start_positions[m]] = my_corrections[m]
          if (i>start_positions[m] and i<end_positions[m]):
              my_new_text[i]=""
      
  my_new_text = "".join(my_new_text)
  return(my_new_text)


Unzipping /tmp/tmpxnvav2tw.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


#**Paragraphs and Bullet Points Generation** 

In [57]:
def Paragraph_divider(text):
  list_text = list(text)
  divider = 0
  counter = 0
  random_int = 0
  for character in range(len(list_text)):
    if ((list_text[character] == '.' and divider==2)):
      list_text[character] = '.\n'
      divider = 0

    if(list_text[character] == ',' and random_int<42):
      random_int = 2

    if(random_int>=2):
      random_int += 1

    if(random_int>=42 and list_text[character] == ','):
      list_text[character] = '.\n'
      random_int = 0

    if((list_text[character] == '.' and divider!=2)):
      divider += 1
  return listToString2(list_text)


In [ ]:
def bullet_points(text):
  lines = text.split(".\n")
  for i in range(len(lines)):
    lines[i] = "*" + lines[i]
  text = "\n".join(lines)
  return text 


# **Keyword Extraction**

In [58]:
simple_kwextractor = yake.KeywordExtractor()
Key_Phrases = []
Weights = []
#keywords = {'Key Phrase':,'Weight':}
keywords = simple_kwextractor.extract_keywords(text)

#for kw in keywords:
	#print(kw)
def free_user(extracted_keywords):
  return_phrases = []
  free_user_phrases = nsmallest(5, extracted_keywords, key=itemgetter(1)) #Lower Score means Higher Relevance
  for phrase,score in free_user_phrases:
    return_phrases.append(phrase)
  return return_phrases
free_user_top5 = free_user(keywords)
print(free_user_top5)


['Apple', 'Apple Distribution International', 'Music Library Content', 'Apple Music', 'content']


In [59]:
def premium_user(extracted_keywords):
  return_phrases = []
  premium_user_phrases = nsmallest(20, extracted_keywords, key=itemgetter(1)) #Lower Score means Higher Relevance
  for phrase,score in premium_user_phrases:
    return_phrases.append(phrase)
  return return_phrases
premium_user_top20 = premium_user(keywords)
print(premium_user_top20)

['Apple', 'Apple Distribution International', 'Music Library Content', 'Apple Music', 'content', 'SERVICES', 'App Store Content', 'Apple Arcade Apps', 'Apple Paid Subscription', 'Licensed Application', 'Apple Music membership', 'Apple Music Family', 'iCloud Music Library', 'Apple Books Content', 'Apple Distribution', 'Music Library', 'Apple Podcast Subscriptions', 'App', 'Apple Arcade', 'STORE CONTENT App']


In [60]:
def business_account_user(extracted_keywords):
  return_phrases = []
  business_user_input = int(input())
  business_user_phrases = nsmallest(business_user_input, extracted_keywords, key=itemgetter(1)) #Lower Score means Higher Relevance
  for phrase,score in business_user_phrases:
    return_phrases.append(phrase)
  return return_phrases
business_user_custom = business_account_user(keywords)
print(business_user_custom)

15
['Apple', 'Apple Distribution International', 'Music Library Content', 'Apple Music', 'content', 'SERVICES', 'App Store Content', 'Apple Arcade Apps', 'Apple Paid Subscription', 'Licensed Application', 'Apple Music membership', 'Apple Music Family', 'iCloud Music Library', 'Apple Books Content', 'Apple Distribution']


# **Whole Function**

In [63]:
def complete_operation(text):
  #text = str(input("Enter Your Text:"))
  simple_kwextractor = yake.KeywordExtractor()
  tokens = ttt.tokenize(text)
  tokenized_paragraphs = paragraph_cleaner(tokens)
  model_word = Word2Vec(tokenized_paragraphs, min_count=1,sg=1)
  score=[]
  for index, sentence in enumerate(tokenized_paragraphs):
      i = tokenized_paragraphs.index(sentence)
      meanScore= meanOfWord(model_word,sentence)
      temp = [i,meanScore]
      score.append(temp)
  token_scores = []
  for scores in score:
    token_scores.append(scores[1])
  sorted_tokens = list(zip(tokens,token_scores))
  sorted_tokens.sort(key= lambda a:a[1], reverse=1)

  keywords = simple_kwextractor.extract_keywords(text)
  summarization_list = []
  Original_Text = ''
  Simplified_Text = ''
  print("select 1 for free account, 2 for premium, 3 for business")
  user = int(input())
  if(user == 1):
    free_user_keywords = free_user(keywords) #Keyword_Extraction
    free_user_summarization = relevant_text_ranker(user,sorted_tokens)
    for input_text in free_user_summarization:
      input_text = text_summarizer_free(input_text)
      new_text = input_text
      input_padded, index_list, len_list = process_input(input_text)
      pred_cwi = model_cwi.predict(input_padded)
      pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
      complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
      bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
      for word_to_replace, l_candidates in bert_candidates:
        tuples_word_zipf = []
        for w in l_candidates:
            tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
        tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
        new_text = re.sub(re.escape(word_to_replace), tuples_word_zipf[0][0], new_text) 
      new_text = re.sub("\n\n","\n",new_text)
      new_text = "*" + new_text + "\n"
      Original_Text += input_text
      Simplified_Text += new_text
    print(Simplified_Text)
    grammatically_correct_simplified_text = grammar_corrector(Simplified_Text)
    return user, grammatically_correct_simplified_text

  if(user == 2):
    premium_user_keywords = premium_user(keywords) #Keyword_Extraction
    premium_user_summarization = relevant_text_ranker(user,sorted_tokens)
    for input_text in premium_user_summarization:
      input_text = text_summarizer_premium(input_text)
      new_text = input_text
      input_padded, index_list, len_list = process_input(input_text)
      pred_cwi = model_cwi.predict(input_padded)
      pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
      complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
      bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
      for word_to_replace, l_candidates in bert_candidates:
        tuples_word_zipf = []
        for w in l_candidates:
            tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
        tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
        new_text = re.sub(re.escape(word_to_replace), tuples_word_zipf[0][0], new_text) 
      new_text = re.sub("\n\n","\n",new_text)
      new_text = "*" + new_text + "\n"
      Original_Text += input_text
      Simplified_Text += new_text
    grammatically_correct_simplified_text = grammar_corrector(Simplified_Text)
    return user, grammatically_correct_simplified_text
    
  if(user == 3):
    business_user_keywords = business_account_user(keywords) #Keyword_Extraction
    points,business_user_summarization = relevant_text_ranker(user,sorted_tokens)
    for input_text in business_user_summarization:
      input_text = text_summarizer_business(input_text)
      new_text = input_text
      input_padded, index_list, len_list = process_input(input_text)
      pred_cwi = model_cwi.predict(input_padded)
      pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
      complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
      bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
      for word_to_replace, l_candidates in bert_candidates:
        tuples_word_zipf = []
        for w in l_candidates:
            tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
        tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
        new_text = re.sub(re.escape(word_to_replace), tuples_word_zipf[0][0], new_text) 
      new_text = re.sub("##g",".",new_text)
      new_text = "*" + new_text + "\n"
      Original_Text += input_text
      Simplified_Text += new_text
    print(Simplified_Text)
    grammatically_correct_simplified_text = grammar_corrector(Simplified_Text)
    return user, grammatically_correct_simplified_text
user,final_text = complete_operation(text)


Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  


select 1 for free account, 2 for premium, 3 for business
1


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


*The Services contain proprietary information and information that is owned by Apple and/or its employees and is protected by applicable intellectual law and other laws and Apple reserves the right at any time to change this Agreement and to add new or additional terms or terms on your use of the Services and
*google Music Library wants you to use your matched or uploaded songs, music and music videos acquired from Apple Music, the iTunes Store or aandher site (“google Music Library Content”) You can upload up to 2000 songs the Songs acquired from iTunes Store do and count against this limit the Songs that do and meet certain conditions (for being very large files) or that are and authorized the
*You can use Content from up to five different Apple products on each device and For any Service, you can have up to 10 devices (but for a maximum of 5 computers) signed in with your Apple ID at one time and Each computer must also be authorized using the same Apple ID and music Content require

In [64]:
print(final_text)
  

*The Services contain proprietary information and information that is owned by Apple and/or its employees and is protected by applicable intellectual law and other laws and Apple reserves the right at any time to change this Agreement and to add new or additional terms or terms on your use of the Services and
*google Music Library wants you to use your matched or uploaded songs, music and music videos acquired from Apple Music, the iTunes Store or handler site (“google Music Library Content”) You can upload up to 2000 songs the Songs acquired from iTunes Store do and count against this limit the Songs that do and meet certain conditions (for being very large files) or that are and authorized the
*You can use Content from up to five different Apple products on each device and For any Service, you can have up to 10 devices (but for a maximum of 5 computers) signed in with your Apple ID at one time and Each computer must also be authorized using the same Apple ID and music Content require

In [66]:
def putfullstop(input):
    words = re.findall('and [A-Z]', input)
    for word in words:
      input = input.replace(word,". "+word[-1])
    return input
final_text = putfullstop(final_text)
def removebadconjunctions(text):
  text = text.replace("and\n",".\n")
  text = text.replace("the\n",".\n")
  return text
final_text = removebadconjunctions(final_text)
print(final_text)

*The Services contain proprietary information and information that is owned by Apple and/or its employees and is protected by applicable intellectual law and other laws . Apple reserves the right at any time to change this Agreement and to add new or additional terms or terms on your use of the Services .
*google Music Library wants you to use your matched or uploaded songs, music and music videos acquired from Apple Music, the iTunes Store or handler site (“google Music Library Content”) You can upload up to 2000 songs the Songs acquired from iTunes Store do and count against this limit the Songs that do and meet certain conditions (for being very large files) or that are and authorized .
*You can use Content from up to five different Apple products on each device . For any Service, you can have up to 10 devices (but for a maximum of 5 computers) signed in with your Apple ID at one time . Each computer must also be authorized using the same Apple ID and music Content requires an HDCP 

# **Document Generation**

In [67]:
if (user == 1):
  with open('/content/'+'Free_User'+'.txt', 'w', encoding="utf-8") as f:
        f.write(final_text)
if (user == 2):
  with open('/content/'+'Premium_User'+'.txt', 'w', encoding="utf-8") as f:
        f.write(final_text)
if (user == 3):
  with open('/content/'+'Business_User'+'.txt', 'w', encoding="utf-8") as f:
        f.write(final_text)